In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

In [2]:
# function courtesy of CryptoDataDownload.com
def fetch_daily_data(symbol):
    pair_split = symbol.split('/')  # symbol must be in format XXX/XXX ie. BTC/EUR
    symbol = pair_split[0] + '-' + pair_split[1]
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?granularity=86400'
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close']      # multiply the BTC volume by closing price to approximate fiat volume
                            
        # if we failed to get any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            data.to_csv(f'cb_{pair_split[0] + pair_split[1]}_daily.csv', index=False)
    else:
        print("Did not receieve OK response from Coinbase API")

<strong>Pulling Bitcoin</strong><br>
To get a baseline, first working with Bitcoin.

In [3]:
fetch_daily_data('BTC/USD')

In [4]:
btc = pd.read_csv('cb_BTCUSD_daily.csv', index_col='date', parse_dates=[0])

In [5]:
btc.head()

,unix,low,high,open,close,volume,vol_fiat
date,,,,,,,
2021-01-14,1610582400,36751.11,40127.66,37393.67,39189.98,27697.447568,1.085462e+09
2021-01-13,1610496000,32309.04,37888.00,34035.53,37393.66,39608.737187,1.481116e+09
2021-01-12,1610409600,32500.00,36604.51,35456.89,34038.98,48227.627520,1.641619e+09
2021-01-11,1610323200,30100.00,38273.88,38168.89,35452.59,102503.156728,3.634002e+09
2021-01-10,1610236800,34444.00,41452.12,40257.43,38171.57,43736.570316,1.669494e+09


In [6]:
btc.drop('unix', axis=1, inplace=True)

In [7]:
btc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 2021-01-14 to 2020-03-21
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   low       300 non-null    float64
 1   high      300 non-null    float64
 2   open      300 non-null    float64
 3   close     300 non-null    float64
 4   volume    300 non-null    float64
 5   vol_fiat  300 non-null    float64
dtypes: float64(6)
memory usage: 16.4+ KB


In [8]:
btc.index = pd.to_datetime(btc.index)

In [9]:
btc.head()

,low,high,open,close,volume,vol_fiat
date,,,,,,
2021-01-14,36751.11,40127.66,37393.67,39189.98,27697.447568,1.085462e+09
2021-01-13,32309.04,37888.00,34035.53,37393.66,39608.737187,1.481116e+09
2021-01-12,32500.00,36604.51,35456.89,34038.98,48227.627520,1.641619e+09
2021-01-11,30100.00,38273.88,38168.89,35452.59,102503.156728,3.634002e+09
2021-01-10,34444.00,41452.12,40257.43,38171.57,43736.570316,1.669494e+09


In [10]:
btc = btc.sort_values('date')

In [11]:
def make_features(data):
    data['range'] = data['high'] - data['low']
    data['%change'] = (1 - (data['high'] / data['low'])) * -1
    data['month'] = data.index.month
    data['dayofweek'] = data.index.dayofweek
    data['week_low_mean'] = data['low'].shift().rolling(7).mean()
    data['week_high_mean'] = data['high'].shift().rolling(7).mean()
    data['month_low_mean'] = data['low'].shift().rolling(30).mean()
    data['month_high_mean'] = data['high'].shift().rolling(30).mean()

In [12]:
make_features(btc)

In [14]:
btc.tail()

,low,high,open,close,volume,vol_fiat,range,%change,month,dayofweek,week_low_mean,week_high_mean,month_low_mean,month_high_mean
date,,,,,,,,,,,,,,
2021-01-10,34444.00,41452.12,40257.43,38171.57,43736.570316,1.669494e+09,7008.12,0.203464,1,6,33499.338571,37684.995714,25461.829667,27617.419000
2021-01-11,30100.00,38273.88,38168.89,35452.59,102503.156728,3.634002e+09,8173.88,0.271557,1,0,33847.250000,38633.870000,26023.963000,28389.387000
2021-01-12,32500.00,36604.51,35456.89,34038.98,48227.627520,1.641619e+09,4104.51,0.126293,1,1,34193.250000,39291.997143,26426.510667,29033.338333
2021-01-13,32309.04,37888.00,34035.53,37393.66,39608.737187,1.481116e+09,5578.96,0.172675,1,2,34565.945714,39592.688571,26885.510667,29606.121000
2021-01-14,36751.11,40127.66,37393.67,39189.98,27697.447568,1.085462e+09,3376.55,0.091876,1,3,34416.874286,39719.545714,27330.222667,30224.055000
